<a href="https://colab.research.google.com/github/febigandap/TugasIndividu_TrainDataset/blob/master/DataProcessing_train_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Pemrosesan data menggunakan RRD pada dataset Train**

**1. Instalasi Spark pada Google Colab**

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-eu.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

**2. Import library Spark yang sudah diinstal**

In [3]:
import findspark
findspark.init("spark-2.4.5-bin-hadoop2.7")# SPARK_HOME
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
from pyspark.sql.functions import col, avg

**3. Inisialisasi SparkContext dan SparkSession**

In [4]:
sc = spark.sparkContext
spark = SparkSession(sc)

**4. Load Dataset**

In [5]:
train_df = spark.read.csv("Train.csv", header=True, inferSchema=True)
train_df.printSchema()            #mengecek schema dari dataset


root
 |-- _c0: integer (nullable = true)
 |-- id: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Customer Type: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Type of Travel: string (nullable = true)
 |-- Class: string (nullable = true)
 |-- Flight Distance: integer (nullable = true)
 |-- Inflight wifi service: integer (nullable = true)
 |-- Departure/Arrival time convenient: integer (nullable = true)
 |-- Ease of Online booking: integer (nullable = true)
 |-- Gate location: integer (nullable = true)
 |-- Food and drink: integer (nullable = true)
 |-- Online boarding: integer (nullable = true)
 |-- Seat comfort: integer (nullable = true)
 |-- Inflight entertainment: integer (nullable = true)
 |-- On-board service: integer (nullable = true)
 |-- Leg room service: integer (nullable = true)
 |-- Baggage handling: integer (nullable = true)
 |-- Checkin service: integer (nullable = true)
 |-- Inflight service: integer (nullable = true)
 |-- Cleanline

In [6]:
train_df.show(5)

+---+------+------+-----------------+---+---------------+--------+---------------+---------------------+---------------------------------+----------------------+-------------+--------------+---------------+------------+----------------------+----------------+----------------+----------------+---------------+----------------+-----------+--------------------------+------------------------+--------------------+
|_c0|    id|Gender|    Customer Type|Age| Type of Travel|   Class|Flight Distance|Inflight wifi service|Departure/Arrival time convenient|Ease of Online booking|Gate location|Food and drink|Online boarding|Seat comfort|Inflight entertainment|On-board service|Leg room service|Baggage handling|Checkin service|Inflight service|Cleanliness|Departure Delay in Minutes|Arrival Delay in Minutes|        satisfaction|
+---+------+------+-----------------+---+---------------+--------+---------------+---------------------+---------------------------------+----------------------+-------------+-

**5. Pemrosesan Data**

**5.1 Pemrosesan Menggunakan Transformasi dan Action pada DataFrame**

Menghitung jumlah penumpang kereta pada masing-masing “class” dengan parameter “gender”, dan “satisfaction” penumpang.

Langkahnya adalah:

Melakukan perhitungan jumlah penumpang. Perhitungan dilakukan dengan melakukan grouping berdasarkan jenis kelamin dan tingkat kepuasan penumpang kereta kemudian dihitung jumlah penumpang untuk masing-masing class nya. Proses ini dapat digabung menjadi satu baris kode (lihat di bawah)

In [7]:
train_df.groupby("class", "gender", "satisfaction").count().show()

+--------+------+--------------------+-----+
|   class|gender|        satisfaction|count|
+--------+------+--------------------+-----+
|     Eco|Female|neutral or dissat...|19471|
|Business|  Male|neutral or dissat...| 7475|
|     Eco|  Male|neutral or dissat...|18573|
|     Eco|Female|           satisfied| 4387|
|Business|Female|neutral or dissat...| 7710|
|Business|  Male|           satisfied|17263|
|     Eco|  Male|           satisfied| 4314|
|Eco Plus|  Male|           satisfied|  914|
|Eco Plus|  Male|neutral or dissat...| 2638|
|Eco Plus|Female|           satisfied|  930|
|Eco Plus|Female|neutral or dissat...| 3012|
|Business|Female|           satisfied|17217|
+--------+------+--------------------+-----+



**5.2 Pemrosesan Menggunakan SparkSQL**

"Menghitung jumlah penumpang kereta pada masing-masing “class” dengan parameter “gender”, dan “satisfaction” penumpang." dapat dikerjakan menggunakan beberapa cara, menggunakan fungsi transformations dan actions pada RDD yang telah dikerjakan di 5.1 atau menggunakan DataFrame dan SparkSQL.

In [12]:
#menggunakan PySparkSQL untuk mencari jumlah penumpang berdasarkan gender dan satisfaction pada setiap class
df_result = spark.sql("SELECT Class, Gender, Satisfaction FROM Train")
df_result.groupby("Class", "Gender", "Satisfaction").count().show()

+--------+------+--------------------+-----+
|   Class|Gender|        Satisfaction|count|
+--------+------+--------------------+-----+
|     Eco|Female|neutral or dissat...|19471|
|Business|  Male|neutral or dissat...| 7475|
|     Eco|  Male|neutral or dissat...|18573|
|     Eco|Female|           satisfied| 4387|
|Business|Female|neutral or dissat...| 7710|
|Business|  Male|           satisfied|17263|
|     Eco|  Male|           satisfied| 4314|
|Eco Plus|  Male|           satisfied|  914|
|Eco Plus|  Male|neutral or dissat...| 2638|
|Eco Plus|Female|           satisfied|  930|
|Eco Plus|Female|neutral or dissat...| 3012|
|Business|Female|           satisfied|17217|
+--------+------+--------------------+-----+



hasil menggunakan transformations dan actions pada langkah 5.1 sama dengan hasil menggunakan PySparkSQL pada langkah 5.2

**BEBERAPA MODIFIKASI PROSES DATA YANG DAPAT KITA LAKUKAN UNTUK MEMPEROLEH INFORMASI LEBIH RINCI LAGI**

Menampilkan 10 data Satisfaction penumpang berjenis kelamin pria pada class Economy

In [19]:
df_result = spark.sql("SELECT Class, Gender, Satisfaction FROM Train WHERE Class ='Eco' and gender = 'Male'")
df_result.show(10)

+-----+------+--------------------+
|Class|Gender|        Satisfaction|
+-----+------+--------------------+
|  Eco|  Male|neutral or dissat...|
|  Eco|  Male|neutral or dissat...|
|  Eco|  Male|neutral or dissat...|
|  Eco|  Male|           satisfied|
|  Eco|  Male|neutral or dissat...|
|  Eco|  Male|neutral or dissat...|
|  Eco|  Male|neutral or dissat...|
|  Eco|  Male|neutral or dissat...|
|  Eco|  Male|neutral or dissat...|
|  Eco|  Male|neutral or dissat...|
+-----+------+--------------------+
only showing top 10 rows



Menampilkan 10 data penumpang berjenis kelamin pria dengan tingkat kepuasan "satisfied" pada class Business.

In [36]:
df_result = spark.sql("SELECT Class, Gender, Satisfaction FROM Train WHERE Class ='Business' and gender = 'Male' and Satisfaction = 'satisfied'")
df_result.show(10)

+--------+------+------------+
|   Class|Gender|Satisfaction|
+--------+------+------------+
|Business|  Male|   satisfied|
|Business|  Male|   satisfied|
|Business|  Male|   satisfied|
|Business|  Male|   satisfied|
|Business|  Male|   satisfied|
|Business|  Male|   satisfied|
|Business|  Male|   satisfied|
|Business|  Male|   satisfied|
|Business|  Male|   satisfied|
|Business|  Male|   satisfied|
+--------+------+------------+
only showing top 10 rows



Menampilkan 10 data class penumpang dengan jenis kelamin wanita dan satisfaction neutral or dissatisfied.

In [33]:
df_result = spark.sql("SELECT Class, Gender, Satisfaction FROM Train WHERE gender = 'Female' and Satisfaction = 'neutral or dissatisfied'")
df_result.show(10)

+--------+------+--------------------+
|   Class|Gender|        Satisfaction|
+--------+------+--------------------+
|Business|Female|neutral or dissat...|
|     Eco|Female|neutral or dissat...|
|Business|Female|neutral or dissat...|
|     Eco|Female|neutral or dissat...|
|Eco Plus|Female|neutral or dissat...|
|     Eco|Female|neutral or dissat...|
|     Eco|Female|neutral or dissat...|
|     Eco|Female|neutral or dissat...|
|     Eco|Female|neutral or dissat...|
|Business|Female|neutral or dissat...|
+--------+------+--------------------+
only showing top 10 rows



Menampilkan 10 data satisfaction penumpang wanita pada class Eco

In [37]:
df_result = spark.sql("SELECT Class, Gender, Satisfaction FROM Train WHERE gender = 'Female' and Class = 'Eco'")
df_result.show(10)

+-----+------+--------------------+
|Class|Gender|        Satisfaction|
+-----+------+--------------------+
|  Eco|Female|neutral or dissat...|
|  Eco|Female|neutral or dissat...|
|  Eco|Female|neutral or dissat...|
|  Eco|Female|neutral or dissat...|
|  Eco|Female|neutral or dissat...|
|  Eco|Female|neutral or dissat...|
|  Eco|Female|           satisfied|
|  Eco|Female|neutral or dissat...|
|  Eco|Female|neutral or dissat...|
|  Eco|Female|neutral or dissat...|
+-----+------+--------------------+
only showing top 10 rows



In [38]:
spark.stop()    #digunakan untuk menghentikan SparkSession